# Авиакомпания Россия

## Рабочая зона Сони

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel('ID.xls')
data.head()

In [ ]:
solution = pd.read_excel('Expert.xls')
solution.head()

In [ ]:
flight = data.iloc[0]
flight

In [ ]:
night_start = '22:00'
night_end = '6:00'

In [ ]:
start = flight['Время вылета']
duration = flight['Налет']

In [ ]:
def sum_time(start, duration):
    st = list(map(lambda x: int(x), start.split(':')))
    dur = list(map(lambda x: int(x), duration.split(':')))
    a = [st[0] + dur[0],st[1] + dur[1]]
    if a[1]>= 60:
        a[0] += 1
        a[1] -= 60
    if a[0]>=24:
        a[0] -= 24
    return str(a[0])+':'+str(a[1])

In [ ]:
def in_minuts(duration):
    dur  = np.array(list(map(lambda x: int(x), duration.split(':'))))
    return dur[0]*60 + dur[1]

In [ ]:
def in_hours(minuts):
    h = int(minuts // 60)
    m = int(minuts % 60)
    if h < 10:
        h = '0'+ str(h)
    else:
        h = str(h)
    if m < 10:
        m = '0'+ str(m)
    else:
        m = str(m)
    return h + ':' + m

In [ ]:
def find_night_minuts(fligt_time):
    night_h1 = list(map(lambda x: in_minuts(x), ['00:00','06:00']))
    night_h2 = list(map(lambda x: in_minuts(x), ['22:00','23:59']))
    f = list(map(lambda x: in_minuts(x), fligt_time))
    delta = 0

    if f[0] < night_h1[1]:
        if f[1] < night_h1[1]: # весь полёт попал в первую половину ночи
            delta += f[1] - f[0] # Длительность полёта в минутах
        else:
            delta += 6*60 - f[0] 
    elif f[0] > night_h1[1] and f[0] < night_h2[0]:
        if f[1] > night_h1[1] and f[1] < night_h2[0]:
            if f[0] < f[1]:
                delta += 0  #дневное время
            else:
                delta += night_h1[1] + night_h2[1] - night_h2[0] + 1#очень длинный перелёт

        elif f[1] > night_h2[0] and f[1] < night_h2[1]:
            delta += f[1] - night_h2[0] #часть полёта в вечернее время

        elif f[1] > night_h1[0] and f[1] < night_h1[1]:
            delta += f[1] + night_h2[1] - night_h2[0] + 1

    elif f[0] > night_h2[0]:
        if f[1] > night_h2[0] and f[1] < night_h2[1]:
            delta += f[1]-f[0]
        elif f[1] < night_h1[1]: 
            delta += night_h2[1] - f[0] + f[1] + 1
        elif f[1] > night_h1[1] and f[1] < night_h2[0]:
            delta += night_h2[1] - f[0] + night_h1[1] + 1

    return delta    

In [ ]:
def night_percent(start, duration): # start и duration 'hh:mm'
    d0 = in_minuts(duration)
    d = in_hours(d0/2)
    t = sum_time(start, d)
    h1 = [start, t]
    t = sum_time(t, '01:30')
    h2 = [t, sum_time(t,d)]
    return (find_night_minuts(h1) + find_night_minuts(h2))/d0

In [ ]:
def is_night_flight(flight):
    start = flight['Время вылета']
    duration = flight['Налет']
    n_pc = night_percent(start, duration)
    if n_pc > 0.5:
        return True
    else:
        return False

In [ ]:
is_night_flight(flight)

## Рабочая зона Лёши

In [1]:
import pandas as pd
import numpy as np

import json

from time import time

from russiya_airlines_solver import algorithm_A2

In [2]:
with open("ideal.json", "r") as file:
    ideal = np.array(json.loads(file.read()))
    
with open("delta.json", "r") as file:
    delta = np.array(json.loads(file.read()))
    
weights = np.ones(ideal.shape[0]) / ideal.shape[0]

In [3]:
value, dist = algorithm_A2(delta, weights, ideal)
print('\nЗначение целевой функции:\t{:.2f}'.format(value))

Время работы функции:	0.3850 сек

Значение целевой функции:	3.94


In [4]:
for k in range(len(dist)):
    print('\nКоличество связок на {}-ом рабочем столе: {}'.format(k + 1, len(dist[k])))


Количество связок на 1-ом рабочем столе: 352

Количество связок на 2-ом рабочем столе: 352

Количество связок на 3-ом рабочем столе: 360

Количество связок на 4-ом рабочем столе: 357

Количество связок на 5-ом рабочем столе: 352

Количество связок на 6-ом рабочем столе: 362
